In [ ]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
%matplotlib inline

## Logistic Regression (PyTorch)

In [ ]:
n_data = torch.ones(1000, 2)

X0 = torch.normal(2 * n_data, 1)
y0 = torch.zeros(1000)
X1 = torch.normal(-2 * n_data, 1)
y1 = torch.ones(1000)

train_X = np.vstack([X0, X1])
train_y = np.vstack([y0, y1]).reshape(-1, 1)

C1 = np.where(train_y == True)[0]
C0 = np.where(train_y == False)[0]

# Define train_X, train_y for torch
train_X, train_y = torch.from_numpy(train_X).float(), torch.from_numpy(train_y).float() 

print(train_X.shape, train_y.shape)

In [ ]:
plt.figure(figsize = (10,8))
plt.plot(train_X[C1,0], train_X[C1,1], 'ro', alpha = 0.3, label='C1')
plt.plot(train_X[C0,0], train_X[C0,1], 'bo', alpha = 0.3, label='C0')
plt.xlabel(r'$x_1$', fontsize = 15)
plt.ylabel(r'$x_2$', fontsize = 15)
plt.legend(loc = 1, fontsize = 12)
plt.axis('equal')
plt.ylim([-5,5])
plt.show()

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

def load_array(data_arrays, batch_size, is_train = True):
    # Define dataset and dataloader
    dataset = TensorDataset(*data_arrays)
    dataloader = DataLoader(dataset = dataset, 
                            batch_size = batch_size,
                            shuffle = is_train)
    return dataloader

In [ ]:
data_iter = load_array((train_X, train_y), batch_size=len(train_y))

In [ ]:
class LogisticRegressionModel(torch.nn.Module):
    # Define init, forward method
    def __init__(self):
        super(LogisticRegressionModel, self).__init__()
        self.layer = torch.nn.Linear(2, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, inputs):
        outputs = self.layer(inputs)
        return self.sigmoid(outputs)

model_logR = LogisticRegressionModel()

if torch.cuda.is_available():
    train_X, train_y = train_X.cuda(), train_y.cuda()
    model_logR.cuda()

In [ ]:
print(model_logR.layer.weight.data)
print(model_logR.layer.bias.data)

In [ ]:
# Define SGD optimizer 
optimizer_logR = torch.optim.SGD(model_logR.parameters(), lr = 0.05)

In [ ]:
model_logR.parameters

In [ ]:
num_epochs = 200
loss_graph_logR = []

for epoch in range(num_epochs):
    for X, y in data_iter:
        # training with predict, loss, zero_grad, backward, step
        predict_logR = model_logR(train_X)
        loss_logR = torch.nn.functional.binary_cross_entropy(predict_logR, train_y)
        optimizer_logR.zero_grad()
        loss_logR.backward()
        optimizer_logR.step()
    loss_graph_logR.append(torch.nn.functional.binary_cross_entropy(model_logR(train_X), train_y))
plt.plot(loss_graph_logR)
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()

In [ ]:
w1 = model_logR.layer.weight[0][0].item()
w2 = model_logR.layer.weight[0][1].item()
b = model_logR.layer.bias.item()

print(w1, w2, b)

xp = np.arange(-4, 4, 0.01).reshape(-1, 1)
yp = - w1 / w2 * xp - b / w2

train_X, train_y = train_X.cpu(), train_y.cpu()

plt.figure(figsize = (10,8))
plt.plot(train_X[C1,0], train_X[C1,1], 'ro', alpha = 0.3, label='C1')
plt.plot(train_X[C0,0], train_X[C0,1], 'bo', alpha = 0.3, label='C0')
plt.plot(xp, yp, 'g', linewidth = 3, label = 'Logistic Regression')
plt.xlabel(r'$x_1$', fontsize = 15)
plt.ylabel(r'$x_2$', fontsize = 15)
plt.legend(loc = 1, fontsize = 12)
plt.axis('equal')
plt.ylim([-4,4])
plt.show()

## MNIST_MLP

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.nn import functional as F
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [ ]:
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:
train_dataset = datasets.MNIST(root = "../data/MNIST",
                               train = True,
                               download = True,
                               transform = transforms.ToTensor())

test_dataset = datasets.MNIST(root = "../data/MNIST",
                              train = False,
                              transform = transforms.ToTensor())

# Define train_loader, Test_loader
train_loader = DataLoader(dataset=train_dataset,
                          batch_size = BATCH_SIZE,
                          shuffle = True)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size = BATCH_SIZE,
                         shuffle = False)

In [ ]:
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

In [ ]:
pltsize = 1
plt.figure(figsize=(10 * pltsize, pltsize))
for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.axis('off')
    plt.imshow(X_train[i, :, :, :].numpy().reshape(28, 28), cmap = "gray_r")
    plt.title('Class: ' + str(y_train[i].item()))

In [ ]:
class NeuralNet(torch.nn.Module):
    # Define init and forward method
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fclayer1 = torch.nn.Linear(28 * 28, 512)
        self.fclayer2 = torch.nn.Linear(512, 256)
        self.fclayer3 = torch.nn.Linear(256, 10)
        self.dropout_prob = 0.5
        self.batch_norm1 = nn.BatchNorm1d(512)
        self.batch_norm2 = nn.BatchNorm1d(256)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.fclayer1(x)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training, p = self.dropout_prob)
        x = self.fclayer2(x)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training, p = self.dropout_prob)
        x = self.fclayer3(x)
        x = F.log_softmax(x, dim = 1)
        return x

In [ ]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

In [ ]:
model = NeuralNet().to(DEVICE)

# Define optimizer, loss
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

criterion = nn.CrossEntropyLoss()

print(model)

In [ ]:
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        # training with output, loss, zero_grad, backward, step
        output = model(image)
        loss = criterion(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
   
        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item())) 

In [ ]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            # Evaluate with output, test_loss, prediction, correct
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))